# Загрузка библиотек

In [2]:
import random
import imageio
import numpy as np
import pandas as pd
import os
import librosa
import wandb
from dataclasses import dataclass
from collections import defaultdict

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import pairwise_distances

from tqdm import tqdm
import scipy.io.wavfile as wav

import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
plt.rcParams['figure.figsize'] = 10, 6
plt.rcParams['font.size'] = 12

import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Set custom layout for plotly
pio.templates['custom'] = go.layout.Template(
    layout= dict(
        font=dict(size=15),
        title=dict(
            font=dict(size=25),
            x=0.5
        ),
        bargap=0.1,
        width=900,
        height=500,
        autosize=False
    )
)
pio.templates.default = 'plotly+custom'
import seaborn as sns
sns.set()

# Загрузка данных

In [3]:
import kagglehub
sripaadsrinivasan_audio_mnist_path = kagglehub.dataset_download('sripaadsrinivasan/audio-mnist')

print('Data source import complete.')

100%|██████████| 948M/948M [00:09<00:00, 103MB/s]

Extracting files...


Data source import complete.


In [4]:
sripaadsrinivasan_audio_mnist_path

'/root/.cache/kagglehub/datasets/sripaadsrinivasan/audio-mnist/versions/1'

In [5]:
# root = '/kaggle/input/audio-mnist/data'
root = '/root/.cache/kagglehub/datasets/sripaadsrinivasan/audio-mnist/versions/1/data'
n = 60
folders = [os.path.join(root, str(i).zfill(2)) for i in range(1, n+1)]

files = []
for folder in folders:
    files += os.listdir(folder)

In [6]:
X = []
Y = []
for file in files:
    label = file.split("_")[0]
    human = file.split("_")[1]
    X.append(os.path.join(root,human,file))
    Y.append(label)

In [7]:
len(X), len(Y)

(30000, 30000)

# Загрузка функций

In [8]:
torch.cuda.is_available()

True

In [9]:
def compute_pairwise_distances(
        all_features: np.ndarray,
        all_labels: np.ndarray,
        num_classes: int,
        device: str = 'cuda',
        metric: str = 'cosine'  # <- теперь явно используется евклидово расстояние
    ):
    model.eval()
    if not torch.cuda.is_available():
        model.to('cpu')
    else:
        model.to(device)

    # Межклассовые расстояния
    inter_class_dist_matrix = np.zeros((num_classes, num_classes))

    for i in range(num_classes):
        for j in range(num_classes):
            if i == j:
                inter_class_dist_matrix[i, j] = 0
            else:
                feats_i = all_features[all_labels == i]
                feats_j = all_features[all_labels == j]

                if len(feats_i) > 0 and len(feats_j) > 0:
                    dist = pairwise_distances(
                        feats_i,
                        feats_j,
                        metric=metric
                    ).mean()
                    inter_class_dist_matrix[i, j] = dist
                else:
                    inter_class_dist_matrix[i, j] = np.nan

    # Внутриклассовые расстояния
    intra_class_distances = np.zeros(num_classes)

    for i in range(num_classes):
        feats_i = all_features[all_labels == i]
        if len(feats_i) > 1:
            dists = pairwise_distances(feats_i, metric=metric)
            intra_class_distances[i] = dists[np.triu_indices(len(feats_i), k=1)].mean()
        else:
            intra_class_distances[i] = np.nan

    return inter_class_dist_matrix, intra_class_distances

In [10]:
class AudioMNISTDataset(Dataset):
    def __init__(
            self,
            X: list[any],
            Y: list[any],
            target_sr: int = 16000
            ) -> None:
        self.audio = X
        self.labels = Y
        self.target_sr = target_sr
        assert len(self.audio) == len(self.labels)

    def __len__(self) -> int:
        return len(self.audio)

    def get_data(self, file: str) -> np.ndarray:
        data, sr = librosa.load(file, sr=None)
        data = librosa.resample(data, orig_sr=sr, target_sr=self.target_sr)
        data = librosa.util.fix_length(data, size=12000)

        return data

    def __getitem__(self, idx: int) -> tuple[torch.Tensor, torch.Tensor]:
        sample = self.audio[idx]
        sample = self.get_data(sample)
        sample = torch.tensor(sample, dtype=torch.float32).unsqueeze(0)

        label = self.labels[idx]
        label = torch.tensor(int(self.labels[idx]), dtype=torch.long)

        return sample, label

In [12]:
class NonComplex(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        # Feature extraction layers
        self.conv_layer1 = nn.Sequential(
            nn.Conv1d(1, 96, kernel_size=11, stride=4, padding=0),
            nn.BatchNorm1d(96),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=5, stride=3)
        )
        self.conv_layer2 = nn.Sequential(
            nn.Conv1d(96, 256, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=3, stride=2)
        )
        self.conv_layer3 = nn.Sequential(
            nn.Conv1d(256, 384, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm1d(384),
            nn.ReLU()
        )
        self.conv_layer4 = nn.Sequential(
            nn.Conv1d(384, 384, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm1d(384),
            nn.ReLU()
        )
        self.conv_layer5 = nn.Sequential(
            nn.Conv1d(384, 256, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=5, stride=3)
        )

        self.convs = nn.Sequential(
            self.conv_layer1,
            self.conv_layer2,
            self.conv_layer3,
            self.conv_layer4,
            self.conv_layer5,
        )

        # Linear layers
        linear_layer_size: int = 4096
        scaling_factor: int = 1
        # scaling_factor: int = 1.7

        self.linear_fc1 = nn.Sequential(
            nn.Linear(5120, int(linear_layer_size * scaling_factor)),
            nn.ReLU()
        )
        self.linear_fc2 = nn.Sequential(
            nn.Linear(
                int(linear_layer_size * scaling_factor),
                int(linear_layer_size * scaling_factor) // 2
            ),
            nn.ReLU()
        )
        self.linear_fc3 = nn.Sequential(
            nn.Linear(
                int(linear_layer_size * scaling_factor) // 2,
                5120,
            ),
            nn.ReLU()
        )

        self.linears = nn.Sequential(
            self.linear_fc1,
            self.linear_fc2,
            self.linear_fc3,
        )

        # Classification head layer
        self.classification_head = nn.Linear(
            5120,
            TrainConfig.n_labels
        )

    def forward(
            self,
            x: torch.Tensor,
            ) -> tuple[torch.Tensor, ...]:

        out = self.convs(x)  # convlutions
        out_after_conv = out.reshape(
            out.size(0),
            -1
        )  # flatten


        out_after_linear = self.linears(out_after_conv)  # backbone

        """ LOG """
        out_after_linear_log = out_after_linear.detach().clone()
        """ LOG """

        out_final = self.classification_head(out_after_linear)  # cls head

        result = (
            out_final,
            out_after_linear_log
        )

        return result

In [13]:
def train_one_epoch(
        model,
        train_dataloader,
        criterion,
        optimizer,
        device,
        print_interval: int = 50,
        log: bool = True
        ) -> float:
    total_loss: float = 0
    correct_predictions: int = 0
    total_samples: int = 0


    model.train()
    tqdm_loader = tqdm(train_dataloader, initial=1, desc='Training')
    for iteration, (audio, label) in enumerate(tqdm_loader, start=1):
        audio = audio.to(device)
        label = label.to(device)

        optimizer.zero_grad()

        preds, _ = model(audio)
        loss = criterion(preds, label)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        if log and iteration != 0 and iteration % print_interval == 0:
            audio, label = BALANCED_SAMPLE
            audio = audio.to(device)
            label = label.to(device)

            model.eval()
            with torch.no_grad():
                preds, *output_logs = model(audio)

            model.train()

            current_loss = total_loss / iteration
            wandb.log({'avg_batch_loss': current_loss})  # logging
            print(f"\nIteration {iteration}, Average Loss: {current_loss}")

    return total_loss

In [14]:
def evaluate_model(
        model,
        dataloader,
        criterion,
        device,
        phase: str = 'Testing',
        log: bool = True
        ) -> tuple[any, ...]:
    total_loss = 0
    correct_predictions = 0
    total_samples = 0
    all_labels = []
    all_preds = []

    nn_logs: dict[int, list[np.ndarray]] = defaultdict(list)  # save nn output logs

    model.eval()
    with torch.no_grad():
        for audio, label in tqdm(dataloader, desc=phase):
            audio = audio.to(device)
            label = label.to(device)

            preds, *output_logs = model(audio)
            loss = criterion(preds, label)
            total_loss += loss.item()

            for i, output_log in enumerate(output_logs, start=1):
                output_log = output_log.cpu().numpy()
                nn_logs[i].extend(output_log)

            _, predicted_labels = torch.max(preds, 1)
            correct_predictions += (predicted_labels == label).sum().item()
            total_samples += label.size(0)

            all_labels.extend(label.cpu().numpy())
            all_preds.extend(predicted_labels.cpu().numpy())

    f1_micro = f1_score(all_labels, all_preds, average='micro')
    f1_macro = f1_score(all_labels, all_preds, average='macro')

    metrics = {
        'avg_loss': total_loss / len(dataloader),
        'accuracy_top1': correct_predictions / total_samples,
        'f1_micro': f1_micro,
        'f1_macro': f1_macro
    }

    if log:
        print(f"\nEvaluation Results: Average Loss: {metrics['avg_loss']}, Accuracy: {metrics['accuracy_top1']:.4f}, "
              f"F1-Micro: {metrics['f1_micro']:.4f}, F1-Macro: {metrics['f1_macro']:.4f}")

    return metrics, nn_logs, all_labels

# Загружаем доп. функции

In [15]:
def get_model_params_count(model: nn.Module) -> tuple[int, int]:
    all_params_count = sum(p.numel() for p in model.parameters())
    requires_grad_params_count = sum(
        p.numel() for p in model.parameters() if p.requires_grad
    )

    return all_params_count, requires_grad_params_count

In [16]:
def get_dtype(model: nn.Module) -> str:
    param_dtype = str(next(model.parameters()).dtype)

    return param_dtype

In [17]:
def set_seed(seed: int) -> None:
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    random.seed(seed)
    np.random.seed(seed)

In [18]:
RANDOM_STATE = 42
set_seed(RANDOM_STATE)

# Конфиг

In [19]:
@dataclass
class TrainConfig:
    n_epochs: int = 5
    lr: float = 5e-5
    batch_size: int = 64
    momentum: float = 0.9

    n_labels: int = 10
    dataset: str = 'AudioMNIST'
    train_size: float = 0.8
    optimizer: str = 'Adam'


config = TrainConfig()

In [20]:
config.__dict__

{'n_epochs': 5,
 'lr': 5e-05,
 'batch_size': 64,
 'momentum': 0.9,
 'n_labels': 10,
 'dataset': 'AudioMNIST',
 'train_size': 0.8,
 'optimizer': 'Adam'}

In [21]:
run = wandb.init(
    entity='aelyovin',
    project=config.dataset,
    name=f'non_complex_bs_{config.batch_size}_lr_{config.lr}',
    config=config.__dict__
)

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: elyovin (aelyovin) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


# Train Test Split

In [22]:
X_train, X_val, y_train, y_val = train_test_split(
    X,
    Y,
    train_size=config.train_size,
    random_state=RANDOM_STATE,
    stratify=Y,
    shuffle=True
)

X_val, X_test, y_val, y_test = train_test_split(
    X_val,
    y_val,
    test_size=0.5,
    random_state=RANDOM_STATE,
    stratify=y_val,
    shuffle=True
)

In [23]:
print(len(X_train), len(X_val), len(X_test))

24000 3000 3000


In [24]:
pd.Series(y_train).value_counts(dropna=False)

,count
6,2400
5,2400
2,2400
8,2400
4,2400
1,2400
7,2400
3,2400
9,2400
0,2400


In [25]:
train_dataset = AudioMNISTDataset(X_train, y_train)
val_dataset = AudioMNISTDataset(X_val, y_val)
test_dataset = AudioMNISTDataset(X_test, y_test)

In [26]:
train_loader = DataLoader(
    train_dataset,
    batch_size=TrainConfig.batch_size,
    shuffle=False,
    drop_last=False
)

val_loader = DataLoader(
    val_dataset,
    batch_size=TrainConfig.batch_size,
    shuffle=False,
    drop_last=False
)

test_loader = DataLoader(
    test_dataset,
    batch_size=TrainConfig.batch_size,
    shuffle=False,
    drop_last=False
)

In [27]:
it = iter(train_loader)
audio, label = next(it)

In [28]:
audio.shape, label.shape

(torch.Size([64, 1, 12000]), torch.Size([64]))

In [29]:
def get_balanced_batch(
        dataset,
        n_samples_per_class: int = 30,
        n_digits: int = 10  # digits 0-9
        ) -> tuple[torch.Tensor, ...]:
    # Group indices by label
    label_to_indices = {}
    for idx, (_, label) in enumerate(dataset):
        label = label.item()
        if label not in label_to_indices:
            label_to_indices[label] = []
        label_to_indices[label].append(idx)

    # Select samples
    selected_indices = []
    for label in range(n_digits):
        indices = label_to_indices[label]
        selected = np.random.choice(indices, n_samples_per_class, replace=False)
        selected_indices.extend(selected)

    # Shuffle the order
    np.random.shuffle(selected_indices)

    # Create a subset dataset
    subset = torch.utils.data.Subset(dataset, selected_indices)
    loader = DataLoader(
        subset,
        batch_size=n_samples_per_class * n_digits,
        shuffle=False
    )

    return loader, next(iter(loader))

In [30]:
BALANCED_LOADER, BALANCED_SAMPLE = get_balanced_batch(
    test_dataset,
    n_samples_per_class=30
)

In [31]:
BALANCED_SAMPLE[1].unique(return_counts=True)

(tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 tensor([30, 30, 30, 30, 30, 30, 30, 30, 30, 30]))

# Обучение

In [32]:
model = NonComplex()

In [33]:
get_model_params_count(model)

(41068618, 41068618)

In [34]:
get_dtype(model)

'torch.float32'

In [35]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [36]:
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=TrainConfig.lr
)

In [37]:
len(train_loader)

375

In [38]:
val_metrics, start_nn_val_logs, start_labels = evaluate_model(
    model,
    val_loader,
    criterion,
    device,
    phase='Evaluating'
)

Evaluating: 100%|██████████| 47/47 [00:04<00:00, 10.27it/s]


Evaluation Results: Average Loss: 2.3026209080472904, Accuracy: 0.1000, F1-Micro: 0.1000, F1-Macro: 0.0182


In [40]:
metrics_logs: dict[int, dict[str, float]] = dict()


for epoch in range(1, TrainConfig.n_epochs + 1):
    print(f'\nEpoch {epoch}')
    total_loss = train_one_epoch(
        model,
        train_loader,
        criterion,
        optimizer,
        device,
        print_interval=25
    )

    val_metrics, nn_val_logs, labels = evaluate_model(
        model,
        val_loader,
        criterion,
        device,
        phase='Evaluating'
    )

    inter_matrix, intra_matrix = compute_pairwise_distances(
        np.array(nn_val_logs[1]),
        np.array(labels),
        10
    )

    rename_map = {key: key + '_val' for key in val_metrics.keys()}
    val_metrics = {rename_map.get(k, k): v for k, v in val_metrics.items()}

    epoch_metrics = {
        'avg_loss_train': total_loss / len(train_loader),
        'mean_intra_distance_val': np.nanmean(intra_matrix),
        'mean_inter_distance_val': np.nanmean(inter_matrix[inter_matrix != 0])

    }
    epoch_metrics.update(val_metrics)
    wandb.log(epoch_metrics)

    dist_matrices = {
        'inter': inter_matrix,
        'intra': intra_matrix
    }
    epoch_metrics.update(dist_matrices)

    # Save model and metrics
    torch.save(model.state_dict(), 'best_model.pth')
    metrics_logs[epoch] = epoch_metrics


    # Drop if overfitting
    if epoch != 1 and metrics_logs[epoch]['avg_loss_val'] > metrics_logs[epoch - 1]['avg_loss_val']:
        model = NonComplex()
        model.load_state_dict(torch.load('best_model.pth'))
        break


Epoch 1


Training:   7%|▋         | 27/375 [00:05<01:09,  4.98it/s]


Iteration 25, Average Loss: 0.0347860346082598


Training:  14%|█▎        | 51/375 [00:10<01:30,  3.56it/s]


Iteration 50, Average Loss: 0.03196282498305664


...


Iteration 350, Average Loss: 0.015885937877901598


Training: 376it [00:53,  6.97it/s]



Iteration 375, Average Loss: 0.016175812961009797


Evaluating: 100%|██████████| 47/47 [00:03<00:00, 14.63it/s]



Evaluation Results: Average Loss: 0.8312321698411982, Accuracy: 0.8333, F1-Micro: 0.8333, F1-Macro: 0.8228


# Тестовые метрики

In [41]:
model.to(device);

In [42]:
test_metrics, nn_test_logs, test_labels = evaluate_model(
    model,
    test_loader,
    criterion,
    device,
    phase='Testing',
)
test_metrics = {key + '_test': val for key, val in test_metrics.items()}

Testing: 100%|██████████| 47/47 [00:08<00:00,  5.76it/s]


Evaluation Results: Average Loss: 0.8599820105319328, Accuracy: 0.8297, F1-Micro: 0.8297, F1-Macro: 0.8218


In [43]:
test_metrics

{'avg_loss_test': 0.8599820105319328,
 'accuracy_top1_test': 0.8296666666666667,
 'f1_micro_test': 0.8296666666666667,
 'f1_macro_test': 0.8218390890308163}

In [44]:
inter_matrix, intra_matrix = compute_pairwise_distances(
    np.array(nn_test_logs[1]),
    np.array(test_labels),
    10
)
dist_metrics = {
    'mean_intra_distance_test': float(np.nanmean(intra_matrix)),
    'mean_inter_distance_test': float(np.nanmean(inter_matrix[inter_matrix != 0]))
}

In [45]:
test_metrics.update(dist_metrics)

In [46]:
test_metrics

{'avg_loss_test': 0.8599820105319328,
 'accuracy_top1_test': 0.8296666666666667,
 'f1_micro_test': 0.8296666666666667,
 'f1_macro_test': 0.8218390890308163,
 'mean_intra_distance_test': 0.07517625112086535,
 'mean_inter_distance_test': 0.44879992273118763}

In [47]:
wandb.log(test_metrics)

In [48]:
wandb.finish()

accuracy_top1_test,▁
accuracy_top1_val,▇▆█▁
avg_batch_loss,█▆▅▄▄▃▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
avg_loss_test,▁
avg_loss_train,█▂▁▁
avg_loss_val,▂▂▁█
f1_macro_test,▁
f1_macro_val,▇▆█▁
f1_micro_test,▁
f1_micro_val,▇▆█▁
mean_inter_distance_test,▁
